In [40]:
from numba import cuda, jit, prange, njit, int64
import numpy as np

In [53]:
@cuda.jit
def addNeighborhood(data_d, vals_d):
    i = cuda.grid(1)
    if i < data_d.shape[0]:
        val = cuda.atomic.compare_and_swap(data_d[i:], 0, 10)
        vals_d[i] = val

@cuda.jit
def setRanges(data_d):
    i = cuda.grid(1)
    datu = cuda.local.array(2, int64)
    datu[0] = i*2
    datu[1] = i*2+1
    if i < data_d.shape[0] // 2:
        data_d[i*2:i*2+2] = datu[:]
        
def testAtomic():
    data = np.zeros(10, np.int64)
    data[5] = 2
    data_d = cuda.to_device(data)
    vals_d = cuda.device_array(10, np.int64)
    addNeighborhood[1, 32](data_d, vals_d)
    return data_d.copy_to_host(), vals_d.copy_to_host()

def testSetRanges():
    data = np.full(10, -1, np.int64)
    data_d = cuda.to_device(data)
    setRanges[1, 32](data_d)
    return data_d.copy_to_host()

In [54]:
test1 = testAtomic()
test1

(array([10, 10, 10, 10, 10,  2, 10, 10, 10, 10], dtype=int64),
 array([0, 0, 0, 0, 0, 2, 0, 0, 0, 0], dtype=int64))

In [50]:
test2 = testSetRanges()
test2

LoweringError: Failed in nopython mode pipeline (step: nopython mode backend)
[1m[1mNRT required but not enabled
[1m
File "<ipython-input-48-efe1d56e2a7f>", line 14:[0m
[1mdef setRanges(data_d):
    <source elided>
    if i < data_d.shape[0] // 2:
[1m        data_d[i*2:i*2+2] = datu[:]
[0m        [1m^[0m[0m
[0m
[0m[1mDuring: lowering "data_d[$100build_slice.16] = $80binary_subscr.5" at <ipython-input-48-efe1d56e2a7f> (14)[0m